In [1]:
import pandas as pd
import csv
import nltk
import xml.etree.ElementTree as ET
import json
from nltk.util import everygrams

In [16]:
 df1 = pd.read_excel('the 500 FB ads file annotated ')

In [ ]:
df1.head() # check the data 

In [5]:
# can skip this step depends on the data we get
# the purpos of this step is getting the human annotation column (Correct topics)

for index, row in df1.iterrows():
    #print(index)
    #print(row)
    if row['False positive'].lower()=='no' and row['False Negative'].lower()=='no':
        #row['Correct Topic(s)']=row['topic label']
        df1.loc[index,'Correct Topic(s)']= row['topic label']

In [6]:
# import the lexicon from json file lexicon_wz_2020.json 
# this lexicon is made from three files: lexicon.txt, lexiconSJ_Dec5_2017.txt and 
# final_word_lists - wordlists.csv(created by Ben who is the annotator)

# Both lexicon.txt and lexiconSJ_Dec5_2017.txt are .xml format data, and the_new_lexicon.xlsx is an excel
# I combined the lexicon of lexicon.txt with the "govenance" lexicon which come from lexiconSJ_Dec5_2017.txt and the_new_lexicon.xlsx
# you can check thoes three files and get some idea about the lexicon

# here we just import the lexicon lexicon_wz_2020.json first

with open ('lexicon_wz_2020.json') as f:
    
    lexicon = json.load(f)



In [17]:
# check the lexicon
# I suggest checking the lexicon under each category, 
# compaire the lexicon of different categories and see whether they have words in common
set(lexicon['Governance'])&set(lexicon['Immigration']) 

set()

In [9]:
# find the words in each message
def find_topic(x,lexicon):
    topics=['Economic', 'Environment', 'Foreign policy', 
            'Governance', 'Immigration', 'Military', 'Safety',
            'Social Programs', 'Social and Cultural']
    token = [w for w in nltk.word_tokenize(x)]
    words = [w for w in token if w.isalpha()]
    # tokenize the ngrams from 1 to 7 words 
    # this step is optional you can use your own way to do tokenizing
    ngrams = [' '.join(w) for w in everygrams(words,min_len=1,max_len=7)] 
    labels = []
    
    for t in topics:
              
        if set(ngrams)&set(lexicon[t]):
            labels.append(t)
            
        else:
            labels.append('nan') 
    
    labels = [x for x in labels if str(x) != 'nan']
    
    if labels:
        
        return ','.join(labels)
    else:
        return "no topic"

In [11]:
# M_label is the label generated 
df1['M_label'] = df1['text'].apply(lambda x: find_topic(x,lexicon))

In [14]:
## make comparison between human annotated data and the label we generated (M_label)
## calculate the kappa score
# label0  human annotated label Correct Topic(s)
# label1 the label that geneated by the lexicon classifier

from sklearn.metrics import cohen_kappa_score

cohen_score1 = cohen_kappa_score(df1['Correct Topic(s)'].astype(str), df1['M_label'].astype(str))
print("aggrement of label 0 and label 1  " + str(cohen_score1))


aggrement of label 0 and label 1  0.4106593782029382
aggrement of label 2 and label 1  0.4106593782029382
